In [ ]:
import pandas as pd
from pubchempy import Compound
from sklearn.preprocessing import LabelEncoder

In [ ]:
%run sample.ipynb
%run pug.ipynb
%run embed_voxel_acellera.ipynb

In [ ]:
rorg_results = pd.read_csv('../data/pubchem/AID_2551_datatable_all.csv', low_memory=False)

In [ ]:
rorg_results = rorg_results[4:].dropna(subset=['PUBCHEM_CID', 'PUBCHEM_SID'])
rorg_results = rorg_results[['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME', 'Phenotype', 'Max_Response']]
rorg_results.PUBCHEM_CID = rorg_results.PUBCHEM_CID.astype(int)

In [ ]:
samp_frac_outcome = {'PUBCHEM_ACTIVITY_OUTCOME':{'Active':.5, 'Inactive':.5}}
rorg_outcome = sample_frac_max(rorg_results[['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME']], samp_frac_outcome)

In [ ]:
samp_frac_phenotype = {'Phenotype':{'Activator':.33, 'Inhibitor':.33, 'Inactive':.33}}
rorg_phenotype = sample_frac_max(rorg_results[['PUBCHEM_CID', 'Phenotype']], samp_frac_phenotype)

In [ ]:
unq_cids = pd.concat([rorg_phenotype.PUBCHEM_CID, rorg_outcome.PUBCHEM_CID]).drop_duplicates()
unq_smis = fetchISMILES(unq_cids)

In [ ]:
unq_smis.PUBCHEM_CID = unq_smis.PUBCHEM_CID.astype(int)
rorg_outcome = pd.merge(rorg_outcome, unq_smis, on='PUBCHEM_CID', how='left')
rorg_phenotype = pd.merge(rorg_phenotype, unq_smis, on='PUBCHEM_CID', how='left')

In [ ]:
lab_encode = LabelEncoder()
rorg_outcome['Y_LABEL'] = lab_encode.fit_transform(rorg_outcome.PUBCHEM_ACTIVITY_OUTCOME)
rorg_phenotype['Y_LABEL'] = lab_encode.fit_transform(rorg_phenotype.Phenotype)

In [ ]:
outcome_grid = voxelize(rorg_outcome.ISOMERIC_SMILES, nconformers=1)
rorg_outcome_X = np.vstack(outcome_grid, axis=1) # TODO:: [UNTESTED] stack conformer dimensions, !!! conformer --> label mappings are lost
phenotype_grid = voxelize(rorg_phenotype.ISOMERIC_SMILES, nconformers=2)
rorg_phenotype_X = np.vstack(phenotype_grid, axis=1) # TODO:: [UNTESTED] stack conformer dimensions, !!! conformer --> label mappings are lost